In [1]:
import pandas as pd
from sodapy import Socrata

This checkout titles class utilizes the Socrata API to access the Seattle Public Library Checkouts by Title dataset.
It even has the option to query the database to grab a subset of data such as only certain months or years or checkouts for books only. After loading the subset of the database, it returns the head of the pandas dataframe. Finally, you have the option to save the subset of data to a csv file locally on your machine.

In [2]:
class checkout_titles:
    def __init__(self, limit=1000, datasource= "data.seattle.gov", ID="tmmm-ytt6", results_df=None):
        
    '''Default to calling the Seattle Public Library Checkouts by title dataset. 
    Other datasets can be called if specified initially.'''
    
        self.limit = limit
        self.datasource = datasource
        self.ID = ID
        self.results_df= results_df
        
    def load_subset(self, limit, query=False):
        '''Uses the sodapy Socrata as an API to access the Seattle Public Library checkouts dataset.'''
        client = Socrata(self.datasource, None)
        if query is not False:
            try:
                results = client.get(self.ID, query=query)
            except HTTPError:
                print('For queries, terms, such as select, groupby, or where,',
                      'need to be lowercase. Strings need to be in quotes',
                      'e.g. "EBOOK" and numbers need to not have quotes.')
            self.results_df = pd.DataFrame.from_records(results)
        else:
            results = client.get("tmmm-ytt6", limit=limit)
            self.results_df = pd.DataFrame.from_records(results)
            
        return self.results_df.head()

    def save_subset(self, filepath, update=False):
        '''Saves the subset of data loaded from the Seattle Public Library Checkouts dataset as csv.'''
        
        if update:
            self.update_df.to_csv(filepath)
            
        else:
            self.results_df.to_csv(filepath)
                                 
    def update_dataset(self, filepaths=[]):
        '''This function concatenates a previous subset or subsets of data into one dataframe
        called update_df. The filepaths variable is a list of CSV filepaths that the user wants concatenated.
        Each CSV file is then loaded into a pandas dataframe, and those dataframes are then combined on the row.

        Note filepaths must be a list with at least 2 elements for the function to work.'''

        df_list = [pd.read_csv(file) for file in filepaths]
        self.update_df = pd.concat(df_list)
        
        return self.update_df.head()


This class has been converted to a [.py file](splapi.py) and utilized to find the checkout information from 2015 to 2021 in this [Jupyter notebook](load_n_save.ipynb). The checkout information was filtered to be from 2016 to 2020 to reduce the number of datapoints since Tableau can only take so up to 15 million. The preliminary visuals from this dataset are [here](https://public.tableau.com/app/profile/louisa.reilly/viz/chkout_spl_16-20/Sheet8).

Below are some examples of using the class in action with small subsets of 1000 rows.

In [3]:
spl = checkout_titles()

In [4]:
spl.load_subset(limit=1000)

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,subjects,creator,publisher,publicationyear
0,Physical,Horizon,SOUNDDISC,2005,5,4,Two dollar bill a Stone Barrington novel,"Mystery fiction, New York N Y Fiction, Barring...",NaN,NaN,NaN
1,Physical,Horizon,SOUNDCASS,2005,5,4,Star witness,"Legal stories, San Francisco Calif Fiction, An...",NaN,NaN,NaN
2,Physical,Horizon,BOOK,2005,5,1,Saint Leibowitz and the wild horse woman a novel,NaN,NaN,NaN,NaN
3,Physical,Horizon,BOOK,2005,5,1,Collector's encyclopedia of depression glass /...,Depression glass Collectors and collecting Cat...,"Florence, Gene, 1944-","Collector Books,",c2000.
4,Physical,Horizon,BOOK,2005,5,1,Cách làm những món ăn đặc biệt ba miền / Lệ-Hoa.,Cooking Vietnamese,Lệ Hoa.,"Đại Nam,",[199-?]


In [5]:
spl.save_subset('/home/jupyter/spl_checkouts2.csv')

Below is an example of using a query.

In [6]:
spl2 = checkout_titles()

In [7]:
spl2.load_subset(limit=1000, query='select * where checkoutyear > 2015')

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,creator,subjects,publisher,publicationyear
0,Digital,OverDrive,EBOOK,2016,10,1,Slim by Design: Mindless Eating Solutions for ...,Brian Wansink,"Health & Fitness, Nonfiction, Sociology",HarperCollins Publishers Inc.,2014
1,Physical,Horizon,VIDEODISC,2016,10,7,Trollflöjten [videorecording] = The magic flut...,NaN,"Operas, Feature films","Criterion Collection,",[2000]
2,Digital,OverDrive,EBOOK,2016,10,1,Great Expectations,Charles Dickens,"Classic Literature, Fiction","Penguin Group (USA), Inc.",2011
3,Physical,Horizon,BOOK,2016,10,1,A dangerous friend / Ward Just.,"Just, Ward S.","Vietnam War 1961 1975 Fiction, Vietnam History...","Houghton Mifflin Co.,",1999.
4,Physical,Horizon,SOUNDDISC,2016,10,3,Bridge School benefit [sound recording].,NaN,"Rock music 2011 2020, Popular music 2011 2020","Reprise,",p2011.


In [8]:
spl2.save_subset('/home/jupyter/checkouts_2021.csv')

Once the CSV file is saved, it is then uploaded into Tableau for some data-vis. Here is the [Tableau workbook](https://public.tableau.com/app/profile/louisa.reilly/viz/spl_checkout_subset/checkouts_per_year?publish=yes). The workbook is still in the development process, and there are plans to build an interactive dashboard.

The Checkouts by Title dataset is updated on the 6th of each month. It can be updated by doing another query for that month, and appending it to that dataset. Through the update_dataset function, each csv file will be specified via a list of filepaths. Those files will be read into pandas dataframes which are then concatenated on the row axis. That larger dataframe can then be saved as CSV file via save_subset if update is set to True.

In [9]:
spl3 = checkout_titles()

In [10]:
spl3.load_subset(limit=10000, query='select * where checkoutyear > 2015 limit 1000')

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,creator,subjects,publisher,publicationyear
0,Digital,OverDrive,EBOOK,2016,10,1,Slim by Design: Mindless Eating Solutions for ...,Brian Wansink,"Health & Fitness, Nonfiction, Sociology",HarperCollins Publishers Inc.,2014
1,Physical,Horizon,VIDEODISC,2016,10,7,Trollflöjten [videorecording] = The magic flut...,NaN,"Operas, Feature films","Criterion Collection,",[2000]
2,Digital,OverDrive,EBOOK,2016,10,1,Great Expectations,Charles Dickens,"Classic Literature, Fiction","Penguin Group (USA), Inc.",2011
3,Physical,Horizon,BOOK,2016,10,1,A dangerous friend / Ward Just.,"Just, Ward S.","Vietnam War 1961 1975 Fiction, Vietnam History...","Houghton Mifflin Co.,",1999.
4,Physical,Horizon,SOUNDDISC,2016,10,3,Bridge School benefit [sound recording].,NaN,"Rock music 2011 2020, Popular music 2011 2020","Reprise,",p2011.


In [11]:
spl3.save_subset('/home/jupyter/spl_checkouts1.csv')

In [12]:
spl3.update_dataset(filepaths = ['/home/jupyter/spl_checkouts1.csv', '/home/jupyter/spl_checkouts2.csv', '/home/jupyter/spl_checkouts3.csv'])

,Unnamed: 0,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,creator,subjects,publisher,publicationyear
0,0,Digital,OverDrive,EBOOK,2016,10,1,Slim by Design: Mindless Eating Solutions for ...,Brian Wansink,"Health & Fitness, Nonfiction, Sociology",HarperCollins Publishers Inc.,2014
1,1,Physical,Horizon,VIDEODISC,2016,10,7,Trollflöjten [videorecording] = The magic flut...,NaN,"Operas, Feature films","Criterion Collection,",[2000]
2,2,Digital,OverDrive,EBOOK,2016,10,1,Great Expectations,Charles Dickens,"Classic Literature, Fiction","Penguin Group (USA), Inc.",2011
3,3,Physical,Horizon,BOOK,2016,10,1,A dangerous friend / Ward Just.,"Just, Ward S.","Vietnam War 1961 1975 Fiction, Vietnam History...","Houghton Mifflin Co.,",1999.
4,4,Physical,Horizon,SOUNDDISC,2016,10,3,Bridge School benefit [sound recording].,NaN,"Rock music 2011 2020, Popular music 2011 2020","Reprise,",p2011.


In [ ]:
spl3.save_subset('/home/jupyter/spl_checkouts_1_to_3.csv', update=True)

The code below is an example of how the corpus could be updated each month.

In [ ]:
spl4 = checkout_titles()

In [ ]:
spl4.load_subset(limit=1000, query='select * where checkoutyear = 2020 and checkoutmonth < 9 limit 1000000')

In [ ]:
spl4.save_subset('/home/jupyter/checkouts2020_1-8.csv')

In [ ]:
spl4.load_subset(limit=1000000, query='select * where checkoutyear = 2020 and checkoutmonth > 8 limit 1000000')

In [ ]:
spl4.save_subset('/home/jupyter/checkouts2020_9-12.csv')

In [ ]:
spl4.update_dataset(filepaths = ['/home/jupyter/checkouts2020_1-8.csv', '/home/jupyter/checkouts2020_9-12.csv'])

In [ ]:
spl4.save_subset('/home/jupyter/checkouts2020_all.csv', update=True)